Two Names, one ID

In [1]:
import wrds
import pandas as pd
db = wrds.Connection()

Loading library list...
Done


In [2]:
# Load data
data = pd.read_csv('data/KEY-Identity-edited.csv')

# Get unique values from issuer column
issuers = data['names'].unique()

# Convert issuers to a list
issuers_list = issuers.tolist()

# Retrieve records from boardex_eur.eur_wrds_org_composition where companyname appears in issuers_list
general_query = f"SELECT * FROM boardex_eur.eur_wrds_org_composition"
germanyinc_query = f"SELECT * FROM boardex_eur.eur_wrds_org_composition WHERE companyname IN {tuple(issuers_list)}"

all_data = db.raw_sql(general_query)
germanyinc_data = db.raw_sql(germanyinc_query)

In [3]:
data

,id,_intid,symbol,names
0,1,2,NaN,UNRESOLVED CASES
1,2,23,1STRED,1st RED AG
2,3,25,AAQ1,aap AG
3,4,27,A4Y,ESTAVIS AG
4,5,28,AGV,AGIV Real Estate AG
...,...,...,...,...
1630,1631,15868,NaN,WACKER CHEMIE
1631,1632,15869,NaN,windeln.de SE
1632,1633,15870,NaN,Zalando SE
1633,1634,15871,NaN,ZhongDe Waste Technology AG


In [4]:
# Group the data by directorid and count unique directornames
director_counts = all_data.groupby('directorid')['directorname'].nunique()

# Filter directorid entries where there are multiple directornames
directors_with_name_changes = director_counts[director_counts > 1].index

# Extract the relevant rows with directorid entries that have name changes
name_change_records = all_data[all_data['directorid'].isin(directors_with_name_changes)]
name_change_records = name_change_records.sort_values(['directorid', 'datestartrole'])

In [5]:
print(len(name_change_records))
print(len(data))

7626
1635


Out of ~775k board member records in the database, there are ~7k records that have an id which has multiple names associated with it. Running this on our specific data yields the following results (NOTE: This is with the data from May, I still need to get a current database snapshot):

In [6]:
# Group the data by directorid and count unique directornames
director_counts = germanyinc_data.groupby('directorid')['directorname'].nunique()

# Filter directorid entries where there are multiple directornames
directors_with_name_changes = director_counts[director_counts > 1].index

# Extract the relevant rows with directorid entries that have name changes
name_change_records = germanyinc_data[germanyinc_data['directorid'].isin(directors_with_name_changes)]
name_change_records = name_change_records.sort_values(['directorid', 'datestartrole'])

In [7]:
name_change_records

,companyid,directorid,directorname,companyname,rolename,datestartrole,dateendrole,seniority
73,3693.0,327007.0,Doctor Wolfgang Plischke,BAYER AG,Division President,2002-01-01,2003-10-01,Senior Manager
42,3693.0,327007.0,Professor Doctor Wolfgang Plischke,BAYER AG,Board Member - Innovation/Technology/Environment,2006-03-01,2014-04-29,Executive Director
322,3693.0,327007.0,Professor Doctor Wolfgang Plischke,BAYER AG,Independent Board Member,2016-04-30,2021-04-27,Supervisory Director
3056,932942.0,2263987.0,Jürgen Böttcher,CROPENERGIES AG,Technical Director,2012-01-01,2020-05-01,Senior Manager
3050,932942.0,2263987.0,JÃ¼rgen BÃ¶ttcher,CROPENERGIES AG,Chief Technical Officer,2020-05-01,None,Executive Director


Based on our data from May, there were only two people who had different names associated with their ID. In the case of Wolfgang Plischke, there is an additional "Professor" title added to his name for his second and third role held at Bayer AG from 2006 to 2014 and 2016 to 2021. Some quick research shows that this is because he started working as an honorary professor at university in 2011. The record for his ongoing position at the time was changed to reflect the update. This means that for WRDS data, the name change can also have occurred within the time that the role was held, not necessarily beforehand.

The second case is an encoder error.